In [1]:
import re
import billboard
import numpy as np

In [2]:
# The current billboard hot 100 is...
chart = billboard.ChartData('hot-100')
print(chart)

hot-100 chart from 2019-11-30
-----------------------------
1. 'Circles' by Post Malone
2. 'Someone You Loved' by Lewis Capaldi
3. 'Good As Hell' by Lizzo
4. 'Memories' by Maroon 5
5. 'Lose You To Love Me' by Selena Gomez
6. 'Senorita' by Shawn Mendes & Camila Cabello
7. 'No Guidance' by Chris Brown Featuring Drake
8. 'everything i wanted' by Billie Eilish
9. '10,000 Hours' by Dan + Shay & Justin Bieber
10. 'Truth Hurts' by Lizzo
11. 'Panini' by Lil Nas X
12. 'Roxanne' by Arizona Zervas
13. 'Trampoline' by SHAED
14. 'HIGHEST IN THE ROOM' by Travis Scott
15. 'Lover' by Taylor Swift
16. 'Woah' by Lil Baby
17. 'Bad Guy' by Billie Eilish
18. 'BOP' by DaBaby
19. 'Dance Monkey' by Tones And I
20. 'Ballin'' by Mustard Featuring Roddy Ricch
21. 'Hot' by Young Thug Featuring Gunna
22. 'One Man Band' by Old Dominion
23. 'Beautiful People' by Ed Sheeran Featuring Khalid
24. 'Only Human' by Jonas Brothers
25. 'Even Though I'm Leaving' by Luke Combs
26. 'Ran$om' by Lil Tecca
27. 'Old Town Road' by 

Let's do a quick function tha extracts the list of unique artists present in a hot-100 chart, including the featured artists. 

In [3]:
def ExtractArtists_inBillboardWeek(chart):
    
    current_artists = [el.artist for el in chart]
    
    # we want to separate the multiple artists specified by "&", "Featuring", "," and "X".
    # we can do this simply by spliting the strings accordingly with the re library.
    current_artists_withFeat = []

    for el in current_artists:
        # separate the names traditionaly defining featuring artists
        split_artists = re.split(', | & | Featuring | featuring | X |Featuring | x |/' , el)
        # append the names one by one to avoid list of lists 
        for el in split_artists:
            current_artists_withFeat.append(el)

            # remove possible void elements
    if "" in current_artists_withFeat:  
        current_artists_withFeat.remove("")
    
    # "unify" and sort the list of artists.
    current_artists_withFeat_uniques = sorted(list(set(current_artists_withFeat)))
    
    return current_artists_withFeat_uniques

In [4]:
# apply the function to the current billboard chart
current_artists_withFeat_uniques = ExtractArtists_inBillboardWeek(billboard.ChartData('hot-100'))
print("Number of artists in current billboard hot-100:",len(current_artists_withFeat_uniques))
current_artists_withFeat_uniques[:10] + ["..."] + current_artists_withFeat_uniques[-10:]

Number of artists in current billboard hot-100: 104


['9lokknine',
 'A Boogie Wit da Hoodie',
 'Ant Saunders',
 'Arizona Zervas',
 'Bad Bunny',
 'Big Sean',
 'Billie Eilish',
 'Billy Ray Cyrus',
 'Blake Shelton',
 'Camila Cabello',
 '...',
 'Trippie Redd',
 'Ty Dolla $ign',
 'Tyga',
 'Wale',
 'Y2K',
 'YNW Melly',
 'Young Thug',
 'YoungBoy Never Broke Again',
 'bbno$',
 'blackbear']

Note that you may have strange names or singular ways of signifying featurings. For example, Lil Nas X having an X in his name makes the function believes it is a feature term. However, it does not really matter: when we will extract albums, "Lil Nas" will just not be found in the Itunes library, and we will skip to the next artist. 

So, we gathered about 100 artists. Even with 10 albums per artists, that makes only about 1000 albums, with possibly overlapping genres. We need more albums, meaning more artists. 

Let us look for the Billboard in previous decades, distant enough to have as many different artists as possible

In [5]:
# let's consider the 1st of january billboard for a few years, every 5 years since 1980
extra_artists  = []
for date in ["".join([str(year),"-01-01"]) for year in [1970+5*i for i in range(10)]]:
    # billboard of this week
    chart = billboard.ChartData('hot-100',date = date)
    # add every artist of this week to the list of extra artists
    extra_temp = ExtractArtists_inBillboardWeek(chart)
    extra_artists = extra_artists + extra_temp
    
extra_artists_uniques = sorted(list(set(extra_artists)))

But it should be enough artists to have a decent amount of albums. We now have 

In [6]:
print(len(extra_artists)+ len(current_artists_withFeat_uniques), "artists")

1153 artists


Which should be enough for a decent amount of albums. 

Important note: You cannot make too many requests in one time, or you'll get an HTTP error. This is unfortunately controled by the server and it does not seem that you can do much about it (although I am not an expert). Therefore, we could try to extract more artists, but it qould require multiple separated queries... 

In [7]:
chart = billboard.ChartData("hot-100",date='2019-11-25')
# try to request the billboard of every week until last week of may ... Until an the HTTP error
while chart.previousDate > '2019-05-30':
    print(chart.date,"...")
    chart =  billboard.ChartData('hot-100', chart.previousDate)

2019-11-30 ...
2019-11-23 ...
2019-11-16 ...
2019-11-09 ...
2019-11-02 ...
2019-10-26 ...
2019-10-19 ...
2019-10-12 ...
2019-10-05 ...
2019-09-28 ...


HTTPError: 429 Client Error: Too Many Requests for url: http://www.billboard.com/charts/hot-100/2019-09-21